In [ ]:
# ✅ 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ✅ 2. Install necessary libraries
!pip install -q transformers datasets accelerate bitsandbytes peft huggingface_hub torch==2.6.0


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# ✅ 3. Import libraries (Cleaned)
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np
from torch.nn.functional import softmax
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
# ✅ 4. Load CSVs from Google Drive
train_df = pd.read_csv("/content/drive/MyDrive/MultiBanFake/Dataset/text/Train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/MultiBanFake/Dataset/text/Validation.csv")
test_df = pd.read_csv("/content/drive/MyDrive/MultiBanFake/Dataset/text/Test.csv")

In [ ]:
# ✅ 5. Combine headline + description into 'text' (Cleaned with strip)
for df in [train_df, val_df, test_df]:
    df["text"] = df["headline"].fillna('').str.strip() + " " + df["description"].fillna('').str.strip()
    df["label"] = df["label"].astype(int)  # Labels: 0=Non-Fake, 1=Fake

print("\n🔍 Sample Data:")
print(train_df.head())


🔍 Sample Data:
               image_id                                           headline  \
0  entertainment_real_1           মুক্তি পেতেই ঝড় তুললো ‘জওয়ান’র নতুন গান!   
1  entertainment_real_2               নতুন এক সিনেমায় চুক্তিবদ্ধ হলেন দীঘি   
2  entertainment_real_3  এবার মায়ের সঙ্গে শুটিংয়ে যাচ্ছে পদ্ম, ভিডিও ভা...   
3  entertainment_real_4     পরিচালক থেকে এবার অভিনেতা মোস্তফা সরয়ার ফারুকী   
4  entertainment_real_5  কলকাতার সায়ন্তিকাকে নিয়ে কক্সবাজারে উড়াল দিলেন...   

                                         description       category  \
0  এবার প্রকাশ্যে এলো শাহরুখের ‘জওয়ান’র ছবিটির তৃ...  entertainment   
1  ইতোমধ্যে বেশ কয়েকটি সিনেমা এবং ওয়েবসিরিজে কাজ ...  entertainment   
2  চিত্রনায়িকা পরীমণি এক মুহূর্তও যেন ছেলেকে চোখে...  entertainment   
3  মোস্তফা সরয়ার ফারুকী প্রায় পঁচিশ বছর ধরে নির্ম...  entertainment   
4  আজ থেকেই সমুদ্রতীরবর্তী শহরে জায়েদ খান ও সায়ন্...  entertainment   

  types of fake news  label                                               text  
0      

In [ ]:
# ✅ 6. Convert to Hugging Face Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df[["text", "label"]]),
    "validation": Dataset.from_pandas(val_df[["text", "label"]]),
    "test": Dataset.from_pandas(test_df[["text", "label"]]),
})

In [ ]:
# ✅ 7. Tokenization
MODEL_NAME = "BanglaLLM/Bangla-s1k-llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id  # Important for batch training

def tokenize_function(example):
    tokens = tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)
    tokens["labels"] = example["label"]
    return tokens

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/7680 [00:00<?, ? examples/s]

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

In [ ]:
# ✅ 8. Load Model with Classification Head + LoRA (No Unsloth)
base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2, torch_dtype=torch.float16, device_map="auto"
)
base_model = prepare_model_for_kbit_training(base_model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)
model = get_peft_model(base_model, peft_config)
model.config.pad_token_id = tokenizer.pad_token_id  # Fix for batch sizes >1
model.print_trainable_parameters()


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at BanglaLLM/Bangla-s1k-llama-3.2-3B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 4,593,664 || all params: 3,217,349,632 || trainable%: 0.1428


In [ ]:
# ✅ 9. Training Setup
model_save_path = "/content/drive/MyDrive/MultiBanFake/BanglaLLM/llama-3.2-3b-instruct-classification"

training_args = TrainingArguments(
    output_dir=model_save_path,
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=2,
    logging_steps=10,
    fp16=True,
    report_to="none",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)


<ipython-input-10-7f639b1e688e>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# ✅ 10. Train the model
trainer.train()

Step,Training Loss
10,1.054700
20,0.967200
30,0.865400
40,0.775200
50,0.820700
60,0.750700
70,0.725300
80,0.738700
90,0.694600
100,0.705400


TrainOutput(global_step=480, training_loss=0.6254764308532079, metrics={'train_runtime': 595.2757, 'train_samples_per_second': 25.803, 'train_steps_per_second': 0.806, 'total_flos': 6.6611117555712e+16, 'train_loss': 0.6254764308532079, 'epoch': 2.0})

In [ ]:
# ✅ 11. Predict on Test Set (with softmax probabilities)
model.eval()
pred_logits = []
true_labels = []

test_loader = torch.utils.data.DataLoader(tokenized_datasets["test"], batch_size=4)

for batch in test_loader:
    # Filter only model-acceptable keys
    inputs = {k: v.cuda() for k, v in batch.items() if k in ['input_ids', 'attention_mask', 'position_ids']}
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = softmax(logits, dim=-1).cpu().numpy()
        pred_logits.extend(probs)
        true_labels.extend(batch["labels"].cpu().numpy())

pred_logits = np.array(pred_logits)
true_labels = np.array(true_labels)


In [ ]:
# ✅ 12. Save probabilities and labels for ensemble
df_probs = pd.DataFrame(pred_logits, columns=["prob_nonfake", "prob_fake"])
df_probs["true_label"] = true_labels

save_path = model_save_path + "/test_predictions_softmax.csv"
df_probs.to_csv(save_path, index=False)
print(f"✅ Softmax probabilities saved at: {save_path}")

✅ Softmax probabilities saved at: /content/drive/MyDrive/MultiBanFake/BanglaLLM/llama-3.2-3b-instruct-classification/test_predictions_softmax.csv


In [ ]:
# ✅ 13. Final Evaluation
pred_classes = np.argmax(pred_logits, axis=1)
accuracy = accuracy_score(true_labels, pred_classes)
print(f"🔹 Accuracy: {accuracy:.4f}")

print("\n🔎 Classification Report:")
print(classification_report(true_labels, pred_classes, target_names=["Non-Fake", "Fake"], digits=4))

🔹 Accuracy: 0.6896

🔎 Classification Report:
              precision    recall  f1-score   support

    Non-Fake     0.6685    0.7521    0.7078       480
        Fake     0.7167    0.6271    0.6689       480

    accuracy                         0.6896       960
   macro avg     0.6926    0.6896    0.6884       960
weighted avg     0.6926    0.6896    0.6884       960

